# Improving Access to Clean, Safe Drinking Water in Tanzania
## Model

Kenneth Liao

---

## Background

The UN publishes and reviews a list of least developed countries (LDC) every 3 years. LDCs are “low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.”$^{1}$. Tanzania has been classified as an LDC since the UN published the first list of LDCs in 1971$^{2}$. A common challenge of LDCs is a lack of infrastructure to support the development of the nation, including access to education and healthcare, waste management, and potable water.

According to UNICEF, as of 2017, more than 24 million Tanzanians lacked access to basic drinking water$^{3}$. This corresponds to only 56.7% of the country’s population having access to basic drinking water. Outside of developed urban areas, much of the potable water is accessed via water pumps. 

Taarifa is an open-source platform for crowd-sourced reporting and triaging of infrastructure related issues. Together with the Tanzanian Ministry of Water, data has been collected for thousands of water pumps throughout Tanzania. The goal of this project is to be able to predict the condition of these water pumps to improve maintenance, reduce pump downtime, and ensure basic water access for tens of millions of Tanzanians.

**References**

1. https://www.un.org/development/desa/dpad/least-developed-country-category.html
2. https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/publication/ldc_list.pdf
3. https://washwatch.org/en/countries/tanzania/summary/statistics/


### Problem Description

Predict the operating condition of water pumps in Tanzania given various metadata on each water pump.

### Strategy

The strategy will be to implement a Random Forest model for multiclass classification of the state of water pumps.

### Data

The dataset is provided by Taarifa, together with the Tanzanian Ministry of Water and is hosted by DrivenData.org:

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/

---

We first import the necessary libraries and the cleaned datasets.

In [1]:
import scipy
import pandas as pd
import numpy as np
import geopy
import dask
from geopy.distance import geodesic
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode
from config import credentials
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, confusion_matrix, f1_score, recall_score, accuracy_score, precision_score, auc
from sklearn.tree import export_graphviz
from IPython.display import Image
from subprocess import call
import joblib
from bayes_opt import BayesianOptimization
import plotly.express as px

px.set_mapbox_access_token(credentials['mapbox_token'])

init_notebook_mode(connected=True)

In [2]:
# load datasets
X_train = pd.read_pickle('data/X_train.pkl')
X_test = pd.read_pickle('data/X_test.pkl')
y_train = pd.read_pickle('data/y_train.pkl')
y_test = pd.read_pickle('data/y_test.pkl')
raw_train = pd.read_csv('data/train.csv')
raw_labels = pd.read_csv('data/train-labels.csv')

# save column names
feature_names = joblib.load('data/feature_names.pkl')

# load sparse versions of X_train & X_test
X_train_s = scipy.sparse.load_npz('data/X_train_s.npz')
X_test_s = scipy.sparse.load_npz('data/X_test_s.npz')
X_train_s_scaled = scipy.sparse.load_npz('data/X_train_s_scaled.npz')
X_test_s_scaled = scipy.sparse.load_npz('data/X_test_s_scaled.npz')

In [3]:
# create class label mapping
labels = {0: 'functional', 1: 'functional needs repair', 2: 'non functional'}
labels_list = list(labels.values())

In [4]:
class_counts = dict(y_train.value_counts())
class_counts

{0: 21540, 2: 15366, 1: 2892}

## Measuring Success

In classification models it's typical to plot the receiver operating characteristic (ROC) curves and compute the area under the curve (AUC) to compare the efficacy of various models. This requires computing the sensitivity and specificity of the model which are defined below. Note that specificity is also known as recall.

<br>

\begin{equation*}
Sensitivity | Recall = \frac{True Positive}{(True Positive + False Negative)}
\end{equation*}

<br>

\begin{equation*}
Specificity = \frac{True Negative}{(True Negative + False Positive)}
\end{equation*}

<br>

These are not good metrics for classfication problems with imbalanced data. For example, if the true class comprises 99% of the data with only 1% being false, then simply guessing true for all of the data would give you 0.99 Sensitivity and 0 specificity, making your model look extremely good. However, this could be a big issue if getting that 1% correct is critical, such as in the case of classifying cancer.

For imbalanced data it's therefore more appropriate to compute the precision and recall curves which focus on the true class.

<br>


\begin{equation*}
Precision = \frac{True Positive}{(True Positive + False Positive)}
\end{equation*}

<br>

\begin{equation*}
F1 = \frac{2 * (Precision * Recall)}{(Precision + Recall)}
\end{equation*}

<br>

We're interested in predicting which pumps are functioning normally, which pumps are functioning but need to be repaired, and which pumps are completely non functioning. If a pump is non functional, it requires immediate attention as the population dependent on that water source cannot access clean water. Therefore, it's most critical that we predict this class with high recall. That is, for non functional pumps, we want to minimize the number of pumps we classify as being functional when they are actually non functional (false negatives). Of course if we took this to the extreme and assumed all pumps are non functional, we would have perfect recall but very low precision. This would be impractical because we would have to essentially send surveyors to every pump anyway to check their status, in which case the model is useless. With this in mind, the next step is to try to optimize this model to improve the recall of the non functional group without lowing too much precision.

### Helper Functions

In [ ]:
def score(y_test, y_pred):
    """
    Compute precision, recall, f1 score, and classification rate.
    Classification rate is simply the accuracy of the model over
    all classes.
    """
    classification_rate = sum(y_test==y_pred)/len(y_test)
    
    scores = pd.DataFrame({'precision': precision_score(y_test, y_pred, average=None),
                           'recall': recall_score(y_test, y_pred, average=None),
                           'f1-score': f1_score(y_test, y_pred, average=None), 
                           'classification_rate': classification_rate},
            index=labels_list).T
    
    return scores

In [ ]:
# retrain model with best parameters found
clf = RandomForestClassifier(class_weight='balanced', max_depth=91,
                                 min_samples_leaf=1, 
                                 max_features=0.02, n_estimators=500,
                                 random_state=42, n_jobs=-1)

clf.fit(X_train_s, y_train)

The best model parameters found were a `max_depth` of 91, `max_features` of 0.02, and `min_samples_leaf` of 1. The model was retrained above with these optimial features and the test results are shown below.

In [ ]:
# score the optimized model
y_pred_test = clf.predict(X_test_s)
score(y_test, y_pred_test)

Below are the scores on the training dataset. We can see that the model fit the training data extremely well... The gap between the train scores and the test scores is fairly large, indicating there might be some improvement to be made by reducing overfitting. It could also be that the test split is not properly sampled from the total distribution.

In [ ]:
# score the optimized model
y_pred_train = clf.predict(X_train_s)
score(y_train, y_pred_train)

The hyperparameters max_depth, min_samples_leaf, and max_features all affect overfitting. Since we have already tuned these parameters, the next thing to try is outside of the model, namely when we perform cross-validation. We can check if splitting the data into more or less folds affects the test scores. I'll first test a cv split of 2, since it trains faster :)

### Feature Importance

In [ ]:
feat_importances = pd.DataFrame({'feature': feature_names, 'importance': clf.feature_importances_}).sort_values('importance', ascending=False).reset_index(drop=True)

trace = go.Bar(x=feat_importances.head(10).feature, y=feat_importances.head(10).importance)

layout = go.Layout(title='Feature Importances',
                  yaxis=dict(title='Relative Importance'))

fig = go.Figure([trace], layout)

iplot(fig, filename='feature_importance.html')

The relative importance of the top 10 most important features for the random forest model are shown above. The GPS coordinates have the most predictive power, followed by `quantity_dry`, `gps_height`, and `day_recorded`. Interestingly, the one feature I engineered `years_since_install`, also made the top 10.

Let's take a look at how the three classes are spacially distributed.

In [5]:
status_map = pd.concat([X_train[['latitude','longitude']], y_train.map(labels)], axis=1)

fig = px.scatter_mapbox(status_map, lat="latitude", lon="longitude", 
                        color='status_group',
                        size_max=15, zoom=4.5)

fig.update_layout(legend=dict(orientation='h', x=0, y=1.1))

iplot(fig, filename='status_group_map.html')

In [ ]:
fig = px.histogram(status_map, x="latitude", color="status_group")
fig.show()

In [ ]:
fig = px.histogram(status_map, x="longitude", color="status_group")
fig.show()

The map above certainly shows how the three classes tend to form clusters. This gives me an idea for some new features that could possibly improve the model. From what we've learend in the feature importance data and the map above, it's possible that the status of a given pump can have some dependence on the status of other proximate pumps. We could therefore compute within some defined distance around each pump, the number of functional vs non functional pumps and weight our predictions based on this.

In [ ]:
quantity_map = pd.concat([X_train[['latitude','longitude', 'quantity_dry']], y_train.map(labels)], axis=1)

fig = px.scatter_mapbox(quantity_map, lat="latitude", lon="longitude", 
                        color='quantity_dry',
                        size_max=15, zoom=4.5)
fig.show()

In [ ]:
fig = px.histogram(quantity_map, x="quantity_dry", color="status_group")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

## Failure Analysis

In [ ]:
raw_train.set_index('id', drop=True, inplace=True)
raw_labels.set_index('id', drop=True, inplace=True)
raw_full = pd.concat([raw_train, raw_labels], axis=1)

In [ ]:
raw_full

In [ ]:
misclassified_idx = X_test[(y_test != y_pred_test)].index

In [ ]:
misclassified_pumps = raw_full.loc[misclassified_idx, :]
misclassified_pumps

In [ ]:
fig = px.histogram(misclassified_pumps, x="waterpoint_type", color="status_group")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
fig = px.scatter_mapbox(misclassified_pumps, lat="latitude", lon="longitude", 
                        color='status_group',
                        size_max=15, zoom=4.5)
fig.show()

In [ ]:
fig = px.histogram(misclassified_pumps, x="basin", color="status_group")
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
fig.show()

## Feature Engineering

In [ ]:
def get_nearby_points(point, point_array, threshold=1000):
    nearby=0
    for p2 in point_array:
        if geodesic(point, p2).miles < threshold:
            nearby+=1
    return nearby

In [ ]:
def scan_nearby(df, threshold=50):
    results = {}
    for i in df.index:
        p1 = (df.loc[i, 'latitude'], df.loc[i, 'longitude'])
        point_array = list(zip(df[df.index!=i].latitude, df[df.index!=i].longitude))
        nearby_points = get_nearby_points(p1, point_array, threshold=threshold)
        results[i] = nearby_points
    return results

In [ ]:
non_func = status_map[status_map.status_group=='non functional']
non_func.head()

In [ ]:
# Break the DF into 12 chunks
chunks = np.array_split(non_func, 200)

In [ ]:
chunks[0]

In [ ]:
from joblib import Parallel, delayed

In [ ]:
results = Parallel(n_jobs=-1, backend='multiprocessing')(map(delayed(scan_nearby), chunks[:10]))

In [ ]:
nearby_counts_df = pd.DataFrame.from_dict(nearby_counts, orient='index', columns=['nearby_non_func'])

In [ ]:
nearby_counts_df